In [1]:
import os
import sys
import json
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import help_functions as hf
from matplotlib import pyplot as plt

from CustomBCE import get_custom_loss


Using TensorFlow backend.


In [2]:
i = 100
with open('training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(i)]
config['batch_size'] = 32

In [3]:
train, train_df = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                    batch_size=config['batch_size'],
                    nr_classes=config['nr_classes'],
                    image_dimension=config['image_dimension'])
y_true = hf.get_y_true(shape=(train.samples, len(train.class_indices)), classes=train.classes)


Found 50000 validated image filenames belonging to 31 classes.


In [4]:
positive_samples_per_class = np.sum(y_true, axis=0)
negative_samples_per_class = y_true.shape[0] - positive_samples_per_class
alpha_weights = negative_samples_per_class / positive_samples_per_class
alpha_weights = tf.cast(alpha_weights, tf.float32)

In [5]:
alpha_weights

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([4.1566666e+02, 4.9454086e+01, 3.6369209e+01, 2.4999000e+04,
       1.5573981e+02, 9.4239624e+02, 1.9366851e+00, 1.9220769e+03,
       2.6901785e+01, 1.8223375e+01, 1.2499000e+04, 3.5871222e+02,
       6.4172974e+00, 2.6215790e+02, 4.3003448e+02, 6.7119888e+01,
       2.9401765e+03, 2.6808676e+01, 1.5672871e+02, 5.9454093e+00,
       3.9353468e+00, 4.4945044e+02, 1.5721488e+00, 8.7967972e+01,
       1.7860807e+01, 3.2992260e+00, 8.5935813e-01, 8.4645764e+02,
       1.5501650e+01, 1.7288223e+01, 4.3923630e+01], dtype=float32)>

In [6]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
base_model = EfficientNetB2(include_top=False, 
                                    weights='imagenet', 
                                    classes=31,
                                    input_shape=(64, 64, 3))
model = Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(31, activation='sigmoid')
    ])

model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss=get_custom_loss(alpha_weights),
            metrics=[
                tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', threshold=0.5),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(num_thresholds=50, curve='PR', name='pr_auc', multi_label=True),
                tf.keras.metrics.AUC(num_thresholds=50, curve='ROC', name='roc_auc', multi_label=True)
            ])

In [ ]:
model.fit(train, verbose=1, epochs=5)